<a href="https://colab.research.google.com/github/gp1313/EVA4/blob/master/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function                                           # Import all the necessary libraries 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):                                                           # This cell constructs neural network, we define a class Net and inherit super class torch.nn.Module
    def __init__(self):
        super(Net, self).__init__()                                             # We call __init__ method of super class nn.Module
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)                             # input 28x28x1   Output 28x28x32     RF 3 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)                            # input 28x28x32  Output 28x28x64     RF 5 
        self.pool1 = nn.MaxPool2d(2, 2)                                         # input 28x28x64  Output 14x14x64     RF 10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)                           # input 14x14x64  Output 14x14x128    RF 12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)                          # input 14x14x128 Output 14x14x256    RF 14
        self.pool2 = nn.MaxPool2d(2, 2)                                         # input 14x14x256 Output 7x7x256      RF 28 
        self.conv5 = nn.Conv2d(256, 512, 3)                                     # input 7x7x256   Output 5x5x512      RF 30
        self.conv6 = nn.Conv2d(512, 1024, 3)                                    # input 5x5x512   Output 3x3x1024     RF 32
        self.conv7 = nn.Conv2d(1024, 10, 3)                                     # input 3x3x1024  Output 1x1x10       RF 34

    def forward(self, x):                                                       # We define forward method, basically decide networks feedforward operations
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)                                                      # reshape (m, 1, 1, 10) to (m, 10) ==> m is batch_size
        return F.log_softmax(x)                                                 # pass the tensor to output layer softmax, softmax convert activations values to probabilistic scores.

In [8]:
!pip install torchsummary                                                       # Install torch summary package
from torchsummary import summary                                                # import summary          
use_cuda = torch.cuda.is_available()                                            # check if cuda is available
device = torch.device("cuda" if use_cuda else "cpu")                            # set device accordingly
model = Net().to(device)                                                        # transfer network to device memory  
summary(model, input_size=(1, 28, 28))                                          # print summary of network defined above    

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)                                                            # set randome seed to 1    
batch_size = 128                                                                # set batch size to 128, i.e 128 images will be processed in a single feedforard    

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}             # set n.o of cpu workers at preprocessing and pin_memory argument to True through kwarg 
                                                                                # In python kwarg allows you to pass list of arguments with keywords) 
train_loader = torch.utils.data.DataLoader(                                     # Initialize train data loader by seting the source data to MNIST dataset, batch size to 128, set shuffle to True and pass kwarg to set above parameters
    datasets.MNIST('../data', train=True, download=True,                        # We call MNIST method from dataset module of torchvision, it provides MNIST images by downloading them into local drive.
                    transform=transforms.Compose([                              # We also use transform method to normalize images by mean and std parameters
                        transforms.ToTensor(),                                  # Images are numpy ndarrays they need to become torch tensors to process them inside a neural network                  
                        transforms.Normalize((0.1307,), (0.3081,))              # Normalize images by substracting mean value 0.1307 and divide by std value 0.3081  
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(                                      # Repeat everything from train loader steps except set the train flag to false, this will effective just download different set of
    datasets.MNIST('../data', train=False, transform=transforms.Compose([       # images and keep everything else same (especially preprocessing part!)  
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)                              # shuffle or no shuffle doesn't matter


In [0]:
from tqdm import tqdm                                                           # Import tqdm package
def train(model, device, train_loader, optimizer, epoch):                       # Define train method 
    model.train()                                                               # Network has to turn train mode on. This affects all the layers such as BN or dropout that behave differently while training and testing  
    pbar = tqdm(train_loader)                                                   # let tqdm wrap around the data loader iterator 
    for batch_idx, (data, target) in enumerate(pbar):                           # Start fetching batches of training images along with ground truth. 
        data, target = data.to(device), target.to(device)                       # Transfer the data to cuda memeory (if available)      
        optimizer.zero_grad()                                                   # Set gradients of all the parameters to zero
        output = model(data)                                                    # Pass batch of input images to network for the feedforward and get output layer tensor
        loss = F.nll_loss(output, target)                                       # Pass output layer tensor and target values to loss function and get loss value tensor
        loss.backward()                                                         # Back propogate from loss value tensor to all the way back to image as tensor in order to computer gradients for each parameters.
        optimizer.step()                                                        # Optimizer will update all the parameters according to the calculated gradients
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')  # Set loss values and batch index to display on status bar

def test(model, device, test_loader):                                           # Define test method
    model.eval()                                                                # Set model to evaluate mode, i.e network will not drop any neurons from fropout layer and BN will use training phase mean and std values.
    test_loss = 0                                                               # Initialize test loss value to zero 
    correct = 0                                                                 # Initialize correct counter to zero
    with torch.no_grad():                                                       # Set grad_required flag of all the tensors in network to zero. 
                                                                                # It is defualt True i.e network stores intermediate values while feedforward so that it can be used while back propogating
                                                                                # to calculate gradients for parameter update. But at the time of eval mode we don't back propogate hene stoing intermediate values is useless

        for data, target in test_loader:                                        # Start fetching batches of test images along with ground truth. 
            data, target = data.to(device), target.to(device)                   # Transfer the data to cuda memeory (if available)     
            output = model(data)                                                # Pass batch of input images to network for the feedforward and get output layer tensor
            test_loss += F.nll_loss(output, target, reduction='sum').item()     # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)                           # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()               # Compute correct estimations from current batch

    test_loss /= len(test_loader.dataset)                                       # aggregate loss value

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
model = Net().to(device)                                                        # Initialize network and transfer in to cuda memory if available.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)                # Initialize SGD optimizer with leaning rate and momentum value.
for epoch in range(1, 2):                                                       # Interate through whole training and testing dataset to train the network, each iteration is known as epoch.
    train(model, device, train_loader, optimizer, epoch)                        # Training the network on whole train dataset.
    test(model, device, test_loader)                                            # Evalute the network on whole eval dataset.

loss=1.951492428779602 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.52it/s]



Test set: Average loss: 1.8770, Accuracy: 2879/10000 (29%)

